In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from splinter import Browser

### NASA Mars News





In [2]:
# Webpage to scrape
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Make request
response = requests.get(url)

# Create soup object
soup = BeautifulSoup(response.text, 'html.parser')

# Collect title and subtitle
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='rollover_description_inner').text

print(news_title)
print(news_p)



NASA's InSight Places First Instrument on Mars



In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.



### JPL Mars Space Images - Featured Image


In [11]:
# Open Browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# Browser visits site
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)

# Browser clicks FULL IMAGE button
browser.click_link_by_partial_text('FULL')

# Create url string variable
page_url = browser.html

# Create soup object
soup = BeautifulSoup(page_url, 'html.parser')

# Collect featured image url

image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]
#Define the base url
base_url = 'https://www.jpl.nasa.gov'
#Combine two url into one
featured_image_url = base_url + image_url

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA20464-1920x1200.jpg


### Mars Weather

* Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.

```python
# Example:
mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'
```

In [19]:
weather_url = 'https://twitter.com/marswxreport?lang=en'
response_w = requests.get(weather_url)
soup = BeautifulSoup(response_w.text, 'html.parser')
results = soup.find('div', class_='stream').ol.find_all('li', class_='js-stream-item stream-item stream-item')

for result in results:
    if result.find('div', class_='js-tweet-text-container').find('p', class_='TweetTextSize').text[:3] == 'Sol':
        mars_weather_tag = result.find('div', class_='js-tweet-text-container').find('p', class_='TweetTextSize')
       
        mars_weather = mars_weather_tag.text.replace(mars_weather_tag.a.text, '')
        break
        
print(mars_weather)

Sol 2308 (2019-02-02), high -13C/8F, low -71C/-95F, pressure at 8.14 hPa, daylight 06:47-18:54


### Mars Facts




In [4]:
url = 'https://space-facts.com/mars/'

tables = pd.read_html(url)
df = tables[0]
df.columns=['Feature', 'info']
df.set_index('Feature', inplace=True)
df.head()


,info
Feature,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"


In [6]:
mars_facts_html_table = df.to_html()
mars_facts_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>info</th>\n    </tr>\n    <tr>\n      <th>Feature</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  <

### Mars Hemispheres


In [3]:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg"},
    {"title": "Cerberus Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg"},
    {"title": "Schiaparelli Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg"},
    {"title": "Syrtis Major Hemisphere", "img_url": "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg"},
]

## Step 2 - MongoDB and Flask Application

Use MongoDB with Flask templating to create a new HTML page that displays all of the information that was scraped from the URLs above.

* Start by converting your Jupyter notebook into a Python script called `scrape_mars.py` with a function called `scrape` that will execute all of your scraping code from above and return one Python dictionary containing all of the scraped data.

* Next, create a route called `/scrape` that will import your `scrape_mars.py` script and call your `scrape` function.

  * Store the return value in Mongo as a Python dictionary.

* Create a root route `/` that will query your Mongo database and pass the mars data into an HTML template to display the data.

* Create a template HTML file called `index.html` that will take the mars data dictionary and display all of the data in the appropriate HTML elements. Use the following as a guide for what the final product should look like, but feel free to create your own design.

- - -

## Step 3 - Submission

To submit your work to BootCampSpot, create a new GitHub repository and upload the following:

1. The Jupyter Notebook containing the scraping code used.

2. Screenshots of your final application.

3. Submit the link to your new repository to BootCampSpot.

## Hints

* Use Splinter to navigate the sites when needed and BeautifulSoup to help find and parse out the necessary data.

* Use Pymongo for CRUD applications for your database. For this homework, you can simply overwrite the existing document each time the `/scrape` url is visited and new data is obtained.

* Use Bootstrap to structure your HTML template.

In [7]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd
import time

def scrape():
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    mars_data_dict = {}

    # Retrieve latest NASA Mars news headline
    nasa_url = 'https://mars.nasa.gov/news/'
    browser.visit(nasa_url)
    response = requests.get(nasa_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_title = soup.find('div', class_="content_title").text
    news_desc = soup.find('div', class_="rollover_description_inner").text

    # Retrieve Mars Images 
    images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(images_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(5)
    browser.click_link_by_partial_text('FULL IMAGE')
    time.sleep(5)
    browser.click_link_by_partial_text('more info')
    time.sleep(5)
    click_thru_html = browser.html
    soup = BeautifulSoup(click_thru_html, 'html.parser')
    featured_image_url = "https://www.jpl.nasa.gov/" + soup.find('figure', class_='lede').a['href']

    # Retrieve Twitter data 
    twitter_url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(twitter_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    mars_weather = soup.find('p', class_="tweet-text").text

    # Retrieve Mars Facts, convert to a dataframe, and save as an html table
    mars_facts_url = "http://space-facts.com/mars/"
    tables = pd.read_html(mars_facts_url)
    mars_facts_df = tables[0]
    mars_facts_df.columns = ['Fact', 'Value']
    mars_facts_df = mars_facts_df.set_index('Fact')
    mars_facts_df.index.name = None
    mars_facts = mars_facts_df.to_html()

    # Manually retrieve Mars Hemispheres images from https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars
    hemisphere_image_urls = [{'title': 'Cerberus Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
          {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
          {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
          {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}
         ]
         
    # Save all data retrieved to a dictionary
    mars_data_dict = {
        "latest_hdl_title": news_title,
        "latest_hdl_description": news_desc,
        "mars_high_res_img": featured_image_url,
        "mars_weather": mars_weather,
        "mars_facts":  mars_facts,
        "mars_imgs": hemisphere_image_urls
    }

    browser.quit()

    return mars_data_dict

In [8]:
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import scrape_mars


# Create an instance of Flask
app = Flask(__name__)

# Use PyMongo to establish Mongo connection
mongo = PyMongo(app, uri="mongodb://localhost:27017/mars_db")

# Route to render index.html template using data from Mongo
@app.route("/")
def home():

    # Find one record of data from the mongo database
    mars_data = mongo.db.mars_data.find_one()

    # Return template and data
    return render_template("index.html", mars=mars_data)

# Route that will trigger the scrape function
@app.route("/scrape")
def scrape():

    # Run the scrape function
    mars_data = scrape_mars.scrape()

    # Update the Mongo database using update and upsert=True
    mongo.db.mars_data.update({}, mars_data, upsert=True)

    # Redirect back to home page
    return redirect("/")

if __name__ == "__main__":
    app.run(debug=True)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

/anaconda3/envs/ut_data/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
